# Digit Recognizer
## Classify handwritten digits using the famous MNIST data

### train.csv
https://www.kaggle.com/c/digit-recognizer/download/train.csv
### test.csv
https://www.kaggle.com/c/digit-recognizer/download/test.csv

The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

In [31]:
# kelas
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import SGD
from keras.utils import np_utils
# basic
import pandas as pd
import numpy as np
from pylab import *

In [2]:
minst = pd.read_csv("./Digit Recognizer/train.csv")

def get_df(minst, train_rate=.75):

    
    minst['is_train'] = np.random.uniform(0, 1, len(minst)) <= train_rate
    train, test = minst[minst['is_train']==True], minst[minst['is_train']==False]
    
    X_label = train.columns[1:-1]
    y_label = train.columns[0]
    
    X_train, y_train = train[X_label], train[y_label]
    X_test, y_test = test[X_label], test[y_label]
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    return np.array(X_train), y_train, np.array(X_test), y_test

X_train, y_train, X_test, y_test = get_df(minst)

# (X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [69]:
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

activation = 'relu'

def network():
    model = Sequential()
    
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], input_shape = (28,28,1)))
    model.add(Activation(activation))
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
    model.add(Activation(activation))
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    
    model.add(Flatten())

    model.add(Dense(128, init='uniform'))
    model.add(Activation(activation))
    model.add(Dropout(0.5))
    model.add(Dense(128, init='uniform'))
    model.add(Activation(activation))
    model.add(Dense(10, init='uniform'))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer="Adadelta",
              metrics=['accuracy'])
    
    return model

In [5]:
tfb = TensorBoard(log_dir='./logs', histogram_freq=0)

In [ ]:
# train
nn = network()
history = nn.fit(X_train, y_train, nb_epoch=40, batch_size=64, verbose=1,callbacks = [tfb], validation_data=(X_test, y_test))
nn.evaluate(X_test, y_test, batch_size=16)

Train on 31529 samples, validate on 10471 samples
Epoch 1/40
31529/31529 [==============================] - 12s - loss: 0.8495 - acc: 0.7055 - val_loss: 0.1637 - val_acc: 0.9497
Epoch 2/40
31529/31529 [==============================] - 12s - loss: 0.2656 - acc: 0.9189 - val_loss: 0.1194 - val_acc: 0.9639
Epoch 3/40
31529/31529 [==============================] - 12s - loss: 0.2108 - acc: 0.9356 - val_loss: 0.1015 - val_acc: 0.9678
Epoch 4/40
31529/31529 [==============================] - 12s - loss: 0.1866 - acc: 0.9424 - val_loss: 0.0892 - val_acc: 0.9719
Epoch 5/40
31529/31529 [==============================] - 12s - loss: 0.1665 - acc: 0.9491 - val_loss: 0.0822 - val_acc: 0.9752
Epoch 6/40
18880/31529 [================>.............] - ETA: 4s - loss: 0.1531 - acc: 0.9524

In [65]:
for layer in nn.layers:
    print(layer.output_shape)

(None, 26, 26, 32)
(None, 26, 26, 32)
(None, 24, 24, 32)
(None, 24, 24, 32)
(None, 12, 12, 32)
(None, 12, 12, 32)
(None, 10, 10, 32)
(None, 10, 10, 32)
(None, 8, 8, 32)
(None, 8, 8, 32)
(None, 4, 4, 32)
(None, 4, 4, 32)
(None, 512)
(None, 128)
(None, 128)
(None, 128)
(None, 128)
(None, 128)
(None, 10)
(None, 10)


In [66]:
test  = pd.read_csv("./Digit Recognizer/test.csv")
_test = np.array(test)
_test = _test.reshape(_test.shape[0],28,28,1)
result = nn.predict(_test)

In [67]:
l = []
for line in result:
    m = np.amax(line)
    for i in range(10):
        if line[i]==m:
            l.append(i)
            
resu = np.array(l)

submit = pd.DataFrame(columns=['ImageId', 'Label'])
submit['Label'] = resu
submit['ImageId'] = [i for i in range(1, len(resu) + 1)]

In [68]:
submit.to_csv('./Digit Recognizer/submit.csv', index=False)